In [1]:
ROOT_DIR= 

series_list= ["Arisa"]
batch_size= 4
IMAGE_DIR= "data/Manga109_released_2023_12_07/images"

In [2]:
from torch.utils.data import DataLoader
from SegDataset import MangaBubbleDataset, gather_json
from SegDataloader import manga_collate_fn, MangaBubbleDataLoader
import json
from torchvision import transforms

mask_dir= 'data/MangaSegmentation/jsons_processed'
train_json = gather_json(series_list, mask_dir)
img_size= (256,256)
TRAIN_JSON_FILE = "train.json"

with open(TRAIN_JSON_FILE, "w+") as f:
    json.dump(train_json, f, indent=2)

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor()])

train_loader = MangaBubbleDataLoader(TRAIN_JSON_FILE, IMAGE_DIR, img_size, batch_size=batch_size, shuffle=False,num_workers=1, transform= transform)

 Processing 1 unique series...

[1/1] Processing series: Arisa
   Total images in JSON: 654
   Total annotations in JSON: 6793
  Images with category 5: 93
  Added 93 images (filtered)
  Added 1212 annotations

   Total images: 93
   Total annotations: 1212
   Images per series (avg): 93.0

 Sample file_names (last 5):
   - Arisa/091.jpg
   - Arisa/092.jpg
   - Arisa/093.jpg
   - Arisa/094.jpg
   - Arisa/095.jpg


In [3]:
from ultralytics import YOLO
YOLO_MODEL_PATH = 'best.pt'
model= YOLO(YOLO_MODEL_PATH)

In [4]:
"""
from EvalSeg import EvalSeg
from tqdm import tqdm
import torch

# Khởi tạo
all_gt_masks = []
all_gt_boxes = []
pred_masks = []
pred_boxes = []

for batch_idx, (imgs, batch_gt_masks, batch_gt_bboxes) in enumerate(tqdm(train_loader.get_loader(), desc="Processing batches")):
    
    for img_idx in range(len(batch_gt_masks)):
        all_gt_masks.append(batch_gt_masks[img_idx])
        all_gt_boxes.append(batch_gt_bboxes[img_idx])  # Đây đã là list của lists [[x1,y1,x2,y2], ...]
    
    results = model.predict(imgs)
    
    for p in results:
        # Boxes cho 1 image - LƯU TOÀN BỘ LIST
        img_pred_boxes = []
        for pb in p.boxes.xyxy.cpu().numpy():
            img_pred_boxes.append(pb.tolist())  # Thêm từng box vào list
        
        # Mask cho 1 image - TẠO COMBINED MASK
        if p.masks is not None:
            binary_masks = p.masks.data > 0.5  # [N, H, W]
            combined_mask = binary_masks.any(dim=0)  # [H, W]
            img_pred_mask = combined_mask.cpu()
        else:
            H, W = imgs.shape[2], imgs.shape[3]
            img_pred_mask = torch.zeros((H, W), dtype=torch.bool)
        
        # Thêm vào list chính
        pred_masks.append(img_pred_mask)           # 1 mask per image
        pred_boxes.append(img_pred_boxes)          # 1 LIST of boxes per image

# Kiểm tra
print(f"Total images: {len(all_gt_masks)}")
print(f"Total GT boxes (lists): {len(all_gt_boxes)}")
print(f"Total GT boxes (individual): {sum(len(boxes) for boxes in all_gt_boxes)}")
print(f"Total Pred boxes (lists): {len(pred_boxes)}")
print(f"Total Pred boxes (individual): {sum(len(boxes) for boxes in pred_boxes)}")

# Sample check
print(f"\nSample GT boxes[0]: {all_gt_boxes[0]}")  # First 2 boxes of first image
print(f"Sample Pred boxes[0]: {pred_boxes[0]}")    # First 2 boxes of first image

eval_seg = EvalSeg(
    gt_masks=all_gt_masks,      # 93 items, mỗi item là tensor [H, W]
    gt_bboxes=all_gt_boxes,     # 93 items, mỗi item là [[x1,y1,x2,y2], [x1,y1,x2,y2], ...]
    pred_masks=pred_masks,      # 93 items, mỗi item là tensor [H, W]
    pred_bboxes=pred_boxes,     # 93 items, mỗi item là [[x1,y1,x2,y2], [x1,y1,x2,y2], ...]
)

print("BBOX METRICS:")
print(eval_seg.eval_bbox())
print("MASK METRICS:")
print(eval_seg.eval_mask())
"""

'\nfrom EvalSeg import EvalSeg\nfrom tqdm import tqdm\nimport torch\n\n# Khởi tạo\nall_gt_masks = []\nall_gt_boxes = []\npred_masks = []\npred_boxes = []\n\nfor batch_idx, (imgs, batch_gt_masks, batch_gt_bboxes) in enumerate(tqdm(train_loader.get_loader(), desc="Processing batches")):\n\n    for img_idx in range(len(batch_gt_masks)):\n        all_gt_masks.append(batch_gt_masks[img_idx])\n        all_gt_boxes.append(batch_gt_bboxes[img_idx])  # Đây đã là list của lists [[x1,y1,x2,y2], ...]\n\n    results = model.predict(imgs)\n\n    for p in results:\n        # Boxes cho 1 image - LƯU TOÀN BỘ LIST\n        img_pred_boxes = []\n        for pb in p.boxes.xyxy.cpu().numpy():\n            img_pred_boxes.append(pb.tolist())  # Thêm từng box vào list\n\n        # Mask cho 1 image - TẠO COMBINED MASK\n        if p.masks is not None:\n            binary_masks = p.masks.data > 0.5  # [N, H, W]\n            combined_mask = binary_masks.any(dim=0)  # [H, W]\n            img_pred_mask = combined_ma

In [5]:

from ClassEvaluator import ClassEvaluator

e= ClassEvaluator(train_loader, model)
res= e.evaluate()
e.print_results()


Evaluating:   0%|                                                                                | 0/24 [00:00<?, ?it/s]


0: 256x256 (no detections), 75.0ms
1: 256x256 6 balloons, 75.0ms
2: 256x256 15 balloons, 75.0ms
3: 256x256 10 balloons, 75.0ms
Speed: 0.0ms preprocess, 75.0ms inference, 17.3ms postprocess per image at shape (1, 3, 256, 256)


Evaluating:   4%|███                                                                     | 1/24 [00:01<00:30,  1.32s/it]


0: 256x256 11 balloons, 89.9ms
1: 256x256 8 balloons, 89.9ms
2: 256x256 17 balloons, 89.9ms
3: 256x256 13 balloons, 89.9ms
Speed: 0.0ms preprocess, 89.9ms inference, 4.1ms postprocess per image at shape (1, 3, 256, 256)


Evaluating:   8%|██████                                                                  | 2/24 [00:01<00:17,  1.27it/s]

...


0: 256x256 16 balloons, 50.0ms
1: 256x256 11 balloons, 50.0ms
2: 256x256 9 balloons, 50.0ms
3: 256x256 2 balloons, 50.0ms
Speed: 0.0ms preprocess, 50.0ms inference, 2.4ms postprocess per image at shape (1, 3, 256, 256)


Evaluating:  92%|█████████████████████████████████████████████████████████████████      | 22/24 [00:13<00:01,  1.94it/s]


0: 256x256 13 balloons, 81.0ms
1: 256x256 4 balloons, 81.0ms
2: 256x256 15 balloons, 81.0ms
3: 256x256 16 balloons, 81.0ms
Speed: 0.0ms preprocess, 81.0ms inference, 2.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 9 balloons, 74.7ms
Speed: 0.0ms preprocess, 74.7ms inference, 2.7ms postprocess per image at shape (1, 3, 256, 256)


Evaluating: 100%|███████████████████████████████████████████████████████████████████████| 24/24 [00:11<00:00,  2.11it/s]


BBox metrics: {'mean_iou': np.float64(0.8903372606837054), 'mAP': np.float64(0.9605954457437866), 'precision': np.float64(0.9395796839408233), 'recall': np.float64(0.8853135306226785), 'f1': np.float64(0.9116392617749134), 'AP@0.5': np.float64(0.9816112075467501), 'precision@0.5': np.float64(0.9816112075467501), 'recall@0.5': np.float64(0.9249174909860416), 'f1@0.5': np.float64(0.9524209099985367), 'AP@0.75': np.float64(0.9395796839408233), 'precision@0.75': np.float64(0.9395796839408233), 'recall@0.75': np.float64(0.8853135306226785), 'f1@0.75': np.float64(0.9116392617749134)}

Mask metrics: {'mean_iou': 0.6859707832336426, 'mean_dice': 0.8088607788085938, 'pixel_precision': 0.7026358246803284, 'pixel_recall': 0.9566082954406738, 'pixel_f1': 0.8088603019714355}

Total images: 93

Total ground truth bboxes: 1212

Total predicted bboxes: 1142

Total predicted masks: 93
